## Constructing A Consistently Oriented Eigenbasis
	
Upon performing SVD, the sign of the basis vectors can be flipped without affecting the validity of the singular value decomposition. That means that if the SVD is performed repeatedly on a slowly evolving system, the derived eigenbasis of the system can abruptly change, from, for example, a left-handed to a right-handed coordinate system. Hence there is a need to find a way to consistently orient the basis of eigenvectors. 

Heuristic techniques can work by taking the dot product between the singular vectors derived for two separate SVDs, and flipping them whenever the dot product is negative. A more rigorous technique was derived by Damask (2019). Damask's technique relies on constructing the sequence of reflection and rotation operations necessary to transform a constituent basis into the derived eigenbasis. The gradual transformation begins with the singular vector with the largest singular value, because it has the greatest numerical certainty. 

Once consistently oriented, directional statistics can be applied to analyze the evolution of the eigensystem over different measurements. 

In general, singular vectors are only labeled by their corresponding singular value (assuming no degeneracy). Rank order changes require attaching an additional label to the eigenvectors.

Reference: 
https://arxiv.org/abs/1912.12983

### Testing the Thucyd Package

Jay's thucyd package can be used to correct sign flips in eigenbases.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import thucyd

In [310]:
### Square Matrices -- doesn't work for complex eigenvalues / eigenvectors 

def check(V,E):
    return np.matmul(np.matmul(V,E),V.T)

for i in range(100):
    N = 20
    M = np.random.randn(N,N)
    E,V = np.linalg.eig(M)

    # get real
    E = np.diag(E.real)
    V = V.real
    M = np.matmul(np.matmul(V,E),V.T)

    # orient the original matrix V
    Vor , Eor , signs , theta_matrix , sort_indices = thucyd.eigen.orient_eigenvectors(V,E)


    assert np.allclose(M,check(Vor,Eor))
    
print("WOrks.")

WOrks.


In [311]:
### SVD -- SVD always returns real eigenvalues bc A^TA is hermitian and real eigenvectors if A is real

### Flipping must be symmetric

def check_svd(U,S,V):
    return np.matmul(np.matmul(U,S),V)

for i in range(100):
    N = 3
    M = np.random.randn(N,N)
    U,S,Vh = np.linalg.svd(M,full_matrices=False)
    S = np.diag(S)

    assert np.all(np.isreal(S))
    assert np.all(S >= 0)
    assert np.all(np.isreal(U))
    assert np.all(np.isreal(V))
    
    Vor , Eor , signs , theta_matrix , sort_indices = thucyd.eigen.orient_eigenvectors(Vh.T,S)
    Vorh = Vor.T
    Uor = np.matmul(U,np.diag(signs))
    
    assert np.allclose(check_svd(U,S,Vh),M)
    assert np.allclose(M,check_svd(Uor,S,Vorh))
    
print("Works.")

Works.


In [ ]:
""" 
Tracking eigenvectors of an evolving system.
"""

